# Laboratorio 7: Solución

## Problema de calendarización de máquinas

En una fábrica existen $\textit{m}$ máquinas que serán utilizadas para procesar $\textit{n}$ trabajos. Cada trabajo debe ser asignado a una sola máquina. La máquina $i \in \{1,\ldots,m\}$ tarda $t_{ij}$ minutos en procesar el trabajo $j\in \{1,\ldots,n\}$. Una vez que una máquina ha iniciado el procesamiento de un trabajo, no es posible interrumpirla hasta que termine con el mismo.

Considerar una instancia con $10$ trabajos y $3$ máquinas. El horizonte de tiempo disponible para el procesamiento es de $120$ minutos. Los tiempos de procesamiento $t_{ij}$ están dados en la tabla:

| $t_{ij}$ |  $1$ |  $2$ |  $3$ |  $4$ |  $5$ |  $6$ |  $7$ |  $8$ |  $9$ | $10$ |
|:--------:|-----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
|   $1$    | $40$ | $60$ | $70$ | $65$ | $60$ | $10$ | $20$ | $35$ | $25$ | $10$ |
|   $2$    | $45$ | $70$ | $65$ | $60$ | $50$ | $15$ | $25$ | $40$ | $20$ | $10$ |
|   $3$    | $50$ | $65$ | $60$ | $70$ | $55$ | $10$ | $20$ | $30$ | $20$ | $15$ |

 
Formular este problema como un **programa lineal entero** para encontrar una asignación de trabajos a máquinas que maximice la cantidad de trabajos procesados dentro de un horizonte de tiempo de $\textit{T}$ minutos. Implementar este programa usando la interfaz Python de Gurobi, resolverlo y mostrar la solución óptima. 

**Formulación del modelo:**

Si suponemos que $I$ es el conjunto de máquinas y $J$ el conjunto de trabajos, usando variables binarias $x_{ij}$ para indicar si la máquina $i$ se le asigna el trabajo $j$, $t_{ij}$ para indicar el tiempo que la máquina $i$ tarda en procesar el trabajo $j$, y $M$ para indicar el horizonte de tiempo disponible para el procesamiento, el problema puede formularse como el siguiente programa lineal entero:


\begin{align*}
&\max \sum_{i \in I} \sum_{j \in J} x_{ij} \\ 
&\mbox{s.r.}\\
& \sum_{i \in I} x_{ij} \leq 1, \quad \forall j \in J, \\
& \sum_{j \in J} t_{ij} x_{ij} \leq M, \quad \forall i \in I, \\
& x_{ij} \in \{0, 1\}, \quad \forall i \in I, j \in J.
\end{align*}


**Implementación del modelo:**

Empezamos por definir los datos del problema en la variable `datos_tiempo`, los conjuntos de índices y el parámetro del horizonte del tiempo $T$:

In [17]:
# Parámetros del modelo
from gurobipy import *

# Tiempos de procesamiento
t = tupledict({(1, 1)  : 40, (1, 2)  : 60, (1, 3)  : 70, (1, 4)  : 65, (1, 5)  : 60,
               (1, 6)  : 10, (1, 7)  : 20, (1, 8)  : 35, (1, 9)  : 25, (1, 10) : 10,
               (2, 1)  : 45, (2, 2)  : 70, (2, 3)  : 65, (2, 4)  : 60, (2, 5)  : 50,
               (2, 6)  : 15, (2, 7)  : 25, (2, 8)  : 40, (2, 9)  : 20, (2, 10) : 10,    
               (3, 1)  : 50, (3, 2)  : 65, (3, 3)  : 60, (3, 4)  : 70, (3, 5)  : 55,
               (3, 6)  : 10, (3, 7)  : 20, (3, 8)  : 30, (3, 9)  : 20, (3, 10) : 15})

# Horizonte de tiempo
T = 120

# Conjuntos de índices
n = 10
m = 3
I = tuplelist(range(1, m+1)) # indexamos las maquinas como {1, ..., m}
J = tuplelist(range(1, n+1)) # indexamos los trabajos como {1, ..., n}


Definimos ahora el objeto modelo y las variables binarias del modelo.

In [18]:
m = Model('machine-scheduling')

# Asignacion máquina-trabajo:
x = m.addVars(I, J, vtype = GRB.BINARY, name="x") # variables indexadas por IxJ.

Añadir la función objetivo: Maximizar la cantidad de trabajos procesados.

In [19]:
# Trabajos procesados
m.setObjective(x.sum('*', '*'), GRB.MAXIMIZE)

Añadir restricciones del modelo.
1. Cada trabajo $j$ puede ser asignado a una sola máquina $i$.

In [20]:
m.addConstrs((x.sum('*', j) <= 1 for j in J), name = "asig") 

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>}

2. Cada máquina debe procesar todos los trabajos que tiene asignados dentro del tiempo límite $T$.

In [21]:
m.addConstrs((x.prod(t, i, '*') <= T for i in I), name = "limite") 

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

Optimizar el modelo:

In [22]:
m.optimize()

Optimize a model with 13 rows, 30 columns and 60 nonzeros
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 9.0000000
Presolve time: 0.00s
Presolved: 13 rows, 30 columns, 60 nonzeros
Variable types: 0 continuous, 30 integer (30 binary)

Root relaxation: objective 1.000000e+01, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   10.00000    0    2    9.00000   10.00000  11.1%     -    0s
H    0     0                      10.0000000   10.00000  0.00%     -    0s

Explored 1 nodes (14 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 10 9 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.0

Mostrar la solución: asignación de trabajos a máquinas

In [23]:
# Asignacion de trabajo a maquina
for j in J:
    for i in I:
        if x[i, j].x >= 0.99:
            print("Trabajo {} asignado a la máquina {}.".format(j, i))

Trabajo 1 asignado a la máquina 1.
Trabajo 2 asignado a la máquina 1.
Trabajo 3 asignado a la máquina 3.
Trabajo 4 asignado a la máquina 2.
Trabajo 5 asignado a la máquina 2.
Trabajo 6 asignado a la máquina 3.
Trabajo 7 asignado a la máquina 1.
Trabajo 8 asignado a la máquina 3.
Trabajo 9 asignado a la máquina 3.
Trabajo 10 asignado a la máquina 2.


In [24]:
print('Maquina\tTrabajos asignados')
print('----------------------')
for i in I:
    print(('{}\t{}'.format(i,[j for j in J if x[i,j].x>=0.99])))

Maquina	Trabajos asignados
----------------------
1	[1, 2, 7]
2	[4, 5, 10]
3	[3, 6, 8, 9]
